In [15]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [16]:
n_years = 1

In [17]:
end_time = datetime.now()
start_time = datetime(
    end_time.year - n_years,
    end_time.month,
    end_time.day
)

In [18]:
def get_train_data(stock_name, start, end, scaler, fit=False):
    df = pdr.get_data_yahoo(
        stock_name,
        start,
        end
    )
    data = df.filter(['Close'])
    dataset = data.values
    
    scaled_data = scaler.fit_transform(dataset) if fit else scaler.transform(dataset)

    X = []
    y = []

    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    
    return X, y

In [19]:
scaler = MinMaxScaler(feature_range=(0,1))

In [20]:
train_stocks = [
    'NYMTN',
    'BIVI',
    'NVMI',
    'CHRD',
    'ASMB',
    'GOGO',
    'CAFG',
    'BVXV',
    'AIBBU',
    'LGVCW'
]

In [21]:
X_train = []
y_train = []

for i, stock in enumerate(train_stocks):
    try:
        X, y = get_train_data(stock, start_time, end_time, scaler, fit=(i==0))
        X_train += X
        y_train += y
    except:
        continue

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [22]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [23]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)),
    LSTM(64, return_sequences=True),
    LSTM(64, return_sequences=False),
    Dense(25),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, batch_size=1, epochs=1)

2023-06-30 23:36:57.982272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-30 23:36:57.984037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-30 23:36:57.985548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1535/1535 [==============================] - 74s 45ms/step - loss: 6.7098


In [24]:
model.save('model1.h5', save_format='h5')